In [3]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [4]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({
            "role": message["role"],
            "content": message["content"]
        })
    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content.strip()

    return response
    
    

In [5]:
client = OpenAI(
            api_key=os.getenv("RUNPOD_TOKEN"),
            base_url=os.getenv("RUNPOD_CHATBOT_URL"),
        )
model_name = os.getenv("MODEL_NAME")

## Get LLM Response

In [6]:
messages = [{"role":"system", "content": "What's the capital of Germany?"},]
response = get_chatbot_response(client, model_name, messages)

In [7]:
response 

'The capital of Germany is Berlin.'

# Prompt Engineering

## Structures Output

In [8]:
system_prompt = """
You are a helpful assistant that answers questions about capital of countries.

Your output should be in a structured JSON format exactly like the one below. You are not allowed to write anything other than the JSON object.
[
{
    "country": the country that you will get teh capital of
    "capital": the capital of teh country stated 
}
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What is the capital of Italy?"})
response = get_chatbot_response(client, model_name, messages)
print(response)


[
{
    "country": "Italy",
    "capital": "Rome"
}
]


In [10]:
type(response)

str

In [11]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

In [12]:
type(json_response[0]),json_response[0]['capital']

(dict, 'Rome')

## Input Structuring

In [13]:
user_input = """
Get me the capital of the following countries:
''' 
1. Italy
2. Germany
3. France
'''
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


In [11]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

## Give the model time to think (Chain of Thoughts)

In [12]:
user_prompt = """
Clculate th eresultof this equation: 1+3

Your output should be in a structured JSON format exactly like the one below. You are not allowed to write anything other than the JSON object.
{
    "result": the final number resulted from calculating the equation above
}
"""

messages = [{"role": "system", "content": user_prompt}]
messages.append({"role": "user", "content": "What is the result of 1+3?"})
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 4
}


In [14]:
259/2*8654+91072*33-12971

4113098.0

In [15]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured JSON format exactly like the one below. You are not allowed to write anything other than the JSON object.
{
    "result": the final number resulted from calculating the equation above
}
"""

messages = [{"role": "system", "content": user_prompt}]
messages.append({"role": "user", "content": "What is the result of 259/2*8654+91072*33-12971?"})
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 2434449
}


In [16]:
4113098.0 - 2434449

1678649.0

In [17]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "steps": "First, calculate the division: 259/2 = 129.5. Then, multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, multiply 91072 by 33: 91072 * 33 = 3,003,336. Now, add 112,141.5 and 3,003,336: 112,141.5 + 3,003,336 = 4,115,477.5. Finally, subtract 12971 from 4,115,477.5: 4,115,477.5 - 12971 = 4,005,806.5.",
    "result": 4005806.5
}


In [18]:
4113098 - 4005806.5

107291.5

# RAG - Retrieval  Augmented Generation

In [19]:
user_prompt = """
What's new in iPhone 16?
"""

messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

There is no such thing as an "iPhone 16." Apple has released the following iPhone models:

* iPhone 12 series (2020)
* iPhone 13 series (2021)
* iPhone 14 series (2022)

If you're looking for information on the latest iPhone models, I can provide you with some general information on the iPhone 14 series. However, please note that there is no iPhone 16.

The iPhone 14 series includes:

* iPhone 14
* iPhone 14 Plus
* iPhone 14 Pro
* iPhone 14 Pro Max

Some of the new features and improvements in the iPhone 14 series include:

* Improved cameras with a new 48MP main camera, a 12MP front camera, and a telephoto lens on the Pro models
* A faster A16 Bionic chip with a 6-core CPU and a 5-core GPU
* Longer battery life, with up to 26 hours of video playback on the iPhone 14 Pro Max
* A new design with a more seamless glass front and a stainless steel frame
* Support for 5G connectivity and Wi-Fi 6
* Improved water resistance with a rating of IP68
* A new feature called "Emergency SOS via Sate

In [20]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [21]:
user_prompt = f"""
{iphone_16}

What's new in iPhone 16? 
"""

messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

* Larger displays: 6.1-inch for the base model, 6.7-inch for the iPhone 16 Plus, 6.3 inches for the iPhone 16 Pro, and 6.9 inches for the iPhone 16 Pro Max
* Thinner bezels and a more durable Ceramic Shield
* New A18 chip (A18 Pro for the Pro models) with enhanced performance, neural engine capabilities, and machine learning tasks
* Upgraded camera systems, including a dual-camera setup with a 48MP main sensor for the base model and a 48MP Ultra Wide and 5x telephoto camera for the Pro models
* "Camera Control" button for more flexible photography options
* Advanced audio features like "Audio Mix" for more refined audio capture during video recording
* Extended battery life, especially in the iPhone 16 Pro Max
* Switch to USB-C for faster charging and data transfer
* Support for up to 2x faster video encoding in the Pro models
* No change in starting prices, with the iPhone 16 starting at $799 and the Pro mo

### Automaticaly extract context from data from database

In [23]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [24]:
data = [iphone_16, samsung_s23]

In [25]:
user_prompt = """
What's new in iPhone 16?
"""

In [26]:
embedding_client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_EMBEDDING_URL')
)

In [27]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(
        input=text_input, 
        model=model_name    
    )
    embeddings = []
    for embedding_object in output.data:
        embeddings.append(embedding_object.embedding)

    return embeddings

In [34]:
user_prompt_embedding = get_embedding(embedding_client, model_name, user_prompt)[0]

In [35]:
user_prompt_embedding

[-0.05107622966170311,
 -0.03489547595381737,
 0.06362394988536835,
 -0.008907047100365162,
 -0.03724626451730728,
 -0.0420089028775692,
 0.015967046841979027,
 0.031155584380030632,
 -0.012799587100744247,
 0.04243631660938263,
 0.05556409806013107,
 0.013288062997162342,
 -0.07217226922512054,
 0.00139673484954983,
 0.09635180979967117,
 0.028697941452264786,
 0.11521918326616287,
 -0.14751963317394257,
 -0.07907198369503021,
 0.016165489330887794,
 -0.03666619956493378,
 -0.014616105705499649,
 -0.039841290563344955,
 -0.030025985091924667,
 0.030025985091924667,
 0.061517395079135895,
 0.0010771268280223012,
 -0.012753792107105255,
 -0.019157402217388153,
 -0.13848282396793365,
 -0.000405234401114285,
 -0.006754701491445303,
 0.01666923053562641,
 0.026835627853870392,
 0.00040475736022926867,
 -0.038497984409332275,
 -0.0031502859201282263,
 0.03492600470781326,
 -0.009632128290832043,
 0.06380712240934372,
 0.029629098251461983,
 0.09103964269161224,
 -0.10141974687576294,
 -0.02

In [36]:
data_embeddings = [get_embedding(embedding_client, model_name, text)[0] for text in data]

In [37]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embeddings)

In [38]:
data_similarity

array([[0.7378098 , 0.52626918]])

In [39]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

0

In [40]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [41]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [42]:
print(user_prompt_with_data)



The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learnin

In [43]:
messages = [{"role": "system", "content": user_prompt_with_data}]
response = get_chatbot_response(client, model_name, messages)
print(response)

According to the text, the new features in iPhone 16 include:

* Larger displays: 6.1-inch for the base model, 6.7-inch for the iPhone 16 Plus, 6.3 inches for the iPhone 16 Pro, and 6.9 inches for the iPhone 16 Pro Max
* Thinner bezels and a more durable Ceramic Shield
* New A18 chip (A18 Pro for the Pro models) with enhanced performance, neural engine capabilities, and machine learning tasks
* Upgraded camera systems, including a dual-camera setup with a 48MP main sensor for the base model and a 48MP Ultra Wide and 5x telephoto camera for the Pro models
* "Camera Control" button for more flexible photography options
* Advanced audio features like "Audio Mix" for refined audio capture during video recording
* Extended battery life, especially in the iPhone 16 Pro Max
* Switch to USB-C for faster charging and data transfer
* Support for up to 2x faster video encoding in the Pro models
* No change in starting prices, with the iPhone 16 starting at $799 and the Pro models starting at $999